In this example we first read a .laz pointcloud file with pylas, then write a .tfrecords file accordingly. We start with importing all required packages:

In [1]:
import tensorflow as tf

import numpy as np
import pylas
import IPython.display as display

Load the .laz point cloud file "Point Clouds/Terrain with Buildings LiDAR.laz":

In [2]:
filename = "Point Clouds/Terrain with Buildings LiDAR.laz"

with pylas.open(filename) as fh:
    print('Points from Header:', fh.header.point_count)
    las = fh.read()
    print(las)
    print('Points from data:', len(las.points))
#     ground_pts = las.classification == 2
#     bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
#     print('Ground Point Return Number distribution:')
#     for r,c in zip(bins,counts):
#         print('    {}:{}'.format(r,c))

Points from Header: 625575
<LasData(1.2, point fmt: <PointFormat(1)>, 625575 points, 0 vlrs)>
Points from data: 625575


The point cloud file contains 625575 points, all stored in pointformat 1. For details of different pointformats, see https://pylas.readthedocs.io/en/latest/intro.html.

In [3]:
lis = list(las.point_format.dimension_names)
extralis = list(las.point_format.extra_dimension_names)
print(lis)
print(extralis)

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time']
[]


The point cloud data dimension names. There are no extra dimension names.

In [4]:
print(np.unique(las.classification))

[2 6]


We define _float_feature() and _int64_feature() to pack list of float or integer into tensorflow features. 
Code is from: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [5]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


Select the point cloud feature we need and make them a dictionary with proper key names:

In [6]:
# Create a dictionary with features that may be relevant.
def pointcloud2tfrecords(las):
  
    position, intensity, user_data, point_source_id, gps_time = [],[],[],[],[]
    
    for p in las.points:
        
        position.append(p['X'])
        position.append(p['Y'])
        position.append(p['Z'])
        intensity.append(p['intensity'])
        user_data.append(p['user_data'])
        point_source_id.append(p['point_source_id'])
#         gps_time.append(p['gps_time'])
    

    feature = {
      'pointcloud/position': _int64_feature(position),
      'pointcloud/intensity': _int64_feature(intensity),
      'pointcloud/user_data': _int64_feature(user_data),
      'pointcloud/point_source_id': _int64_feature(point_source_id),
#       'pointcloud/gps_time': _float_feature(gps_time),
    }
    
    return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(pointcloud2tfrecords(las)).split('\n')[:15]:
    print(line)
print('...')


features {
  feature {
    key: "pointcloud/intensity"
    value {
      int64_list {
        value: 72
        value: 88
        value: 104
        value: 88
        value: 97
        value: 79
        value: 79
        value: 94
        value: 59
        value: 103
...


Write it into a .tfrecords file. 

In [7]:
# Write the point cloud to `Terrain with Buildings LiDAR.tfrecords`.
# First, process the point cloud .laz file into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'Terrain with Buildings LiDAR.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    
    tf_example = pointcloud2tfrecords(las)
    writer.write(tf_example.SerializeToString())

You can now try reading the newly create .tfrecords file using the read_tfrecords.ipynb